In [1]:
%matplotlib inline
import numpy as np
import torch
torch.set_printoptions(edgeitems=2)

In [2]:
t_c = torch.tensor([0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0])
t_u = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4])
t_un = 0.1 * t_u

In [3]:
def model(t_u, w, b):
    return w * t_u + b

In [4]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c) ** 2
    return squared_diffs.mean()

In [6]:
# モデルパラメータ（重み w=1.0, バイアス b=0.0）を初期化
# requires_grad=True により、このテンソルに対する勾配を自動計算するよう設定
params = torch.tensor([1.0, 0.0], requires_grad=True)

In [7]:
params.grad is None

True

In [8]:
# モデルによる予測値と正解値（摂氏）との損失（平均二乗誤差）を計算
loss = loss_fn(model(t_u, *params), t_c)

# 損失関数をモデルパラメータ w, b に関して自動微分（backpropagation）
loss.backward()

# 計算された勾配を確認（params に設定された requires_grad=True により利用可能）
params.grad

tensor([4517.2964,   82.6000])

In [9]:
# 勾配がすでに存在する場合（過去に backward() を呼んだ場合）、
# 次の backward 計算に備えて勾配をゼロで初期化
if params.grad is not None:
    params.grad.zero_()

In [10]:
# 線形モデルの学習ループ
def training_loop(n_epochs, learning_rate, params, t_u, t_c):
    for epoch in range(1, n_epochs + 1):
        # 勾配がすでに計算されている場合はゼロ初期化（勾配の累積を防ぐ）
        if params.grad is not None:
            params.grad.zero_()

        # モデルによる予測値を計算
        t_p = model(t_u, *params)

        # 予測値と正解値との損失を計算（平均二乗誤差）
        loss = loss_fn(t_p, t_c)

        # 誤差をパラメータに対して逆伝播し、勾配を計算
        loss.backward()

        # 勾配を使ってパラメータを更新（計算グラフを無効化してメモリ節約）
        with torch.no_grad():
            params -= learning_rate * params.grad

        # 500エポックごとに損失を出力
        if epoch % 500 == 0:
            print("Epoch %d, Loss %f" % (epoch, float(loss)))

    # 学習後の最終パラメータを返す
    return params

In [11]:
training_loop(
    n_epochs=5000,
    learning_rate=1e-2,
    params=torch.tensor([1.0, 0.0], requires_grad=True),  # <1>
    t_u=t_un,  # <2>
    t_c=t_c,
)

Epoch 500, Loss 7.860118
Epoch 1000, Loss 3.828538
Epoch 1500, Loss 3.092191
Epoch 2000, Loss 2.957697
Epoch 2500, Loss 2.933134
Epoch 3000, Loss 2.928648
Epoch 3500, Loss 2.927831
Epoch 4000, Loss 2.927680
Epoch 4500, Loss 2.927652
Epoch 5000, Loss 2.927647


tensor([  5.3671, -17.3012], requires_grad=True)